In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
import time
import nltk
from nltk.translate.bleu_score import corpus_bleu
import pickle


# Если ещё не скачаны нужные данные NLTK, то раскомментируйте следующую строку:
# nltk.download('punkt')


#########################################
# 1. Формирование датасета и словарей   #
#########################################

class TranslationDataset(Dataset):
    def __init__(self, file_path, max_len=50):
        """
        Читает файл, где каждая строка имеет формат:
          <английский текст><tab><украинский текст>
        Производит базовую токенизацию (split по пробелам) и формирует словари.
        """
        self.src_texts = []  # английские тексты
        self.trg_texts = []  # украинские тексты
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                if "\t" in line:
                    src, trg = line.strip().split("\t")
                    self.src_texts.append(src.lower())
                    self.trg_texts.append(trg.lower())

        # Токенизация (простой split по пробелам)
        self.src_tokenized = [src.split() for src in self.src_texts]
        self.trg_tokenized = [trg.split() for trg in self.trg_texts]

        # Построение словарей (индексация). Специальные токены:
        # <pad>: заполнение, <sos>: начало предложения, <eos>: конец предложения, <unk>: неизвестное слово.
        self.src_vocab = self.build_vocab(self.src_tokenized)
        self.trg_vocab = self.build_vocab(self.trg_tokenized)

        self.max_len = max_len

    def build_vocab(self, tokenized_texts):
        vocab = {'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3}
        idx = 4
        for tokens in tokenized_texts:
            for token in tokens:
                if token not in vocab:
                    vocab[token] = idx
                    idx += 1
        return vocab

    def __len__(self):
        return len(self.src_tokenized)

    def __getitem__(self, idx):
        # Преобразуем исходное предложение в индексы
        src_tokens = self.src_tokenized[idx]
        src_indices = [self.src_vocab.get(token, self.src_vocab['<unk>']) for token in src_tokens]

        # Для целевого языка добавляем <sos> и <eos>
        trg_tokens = self.trg_tokenized[idx]
        trg_indices = [self.trg_vocab['<sos>']] + \
                      [self.trg_vocab.get(token, self.trg_vocab['<unk>']) for token in trg_tokens] + \
                      [self.trg_vocab['<eos>']]

        return torch.tensor(src_indices, dtype=torch.long), torch.tensor(trg_indices, dtype=torch.long)


def collate_fn(batch):
    """
    Формирует батч с паддингом.
    Возвращает:
      - src_batch: [B, max_src_len]
      - trg_batch: [B, max_trg_len]
      - src_lengths: список длин исходных последовательностей (без паддинга)
    """
    src_batch, trg_batch = zip(*batch)
    src_lengths = [len(s) for s in src_batch]
    src_batch = nn.utils.rnn.pad_sequence(src_batch, batch_first=True, padding_value=0)
    trg_batch = nn.utils.rnn.pad_sequence(trg_batch, batch_first=True, padding_value=0)
    return src_batch, trg_batch, src_lengths


#########################################
# 2. Архитектура нейронной сети (Seq2Seq)#
#    с механизмом внимания              #
#########################################

# --- ЭНКОДЕР ---
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers=1):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, batch_first=True)

    def forward(self, src, src_lengths):
        # src: [B, src_len]
        embedded = self.embedding(src)  # [B, src_len, emb_dim]
        # Упаковываем последовательности для LSTM
        packed = nn.utils.rnn.pack_padded_sequence(embedded, src_lengths, batch_first=True, enforce_sorted=False)
        packed_outputs, (hidden, cell) = self.rnn(packed)
        # Восстанавливаем последовательности
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs, batch_first=True)
        # outputs: [B, src_len, hid_dim]
        return outputs, hidden, cell


# --- МОДУЛЬ ВНИМАНИЯ (Attention) ---
class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hid_dim * 2, hid_dim)
        self.v = nn.Linear(hid_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        """
        hidden: [B, hid_dim] — текущее состояние декодера (последний слой)
        encoder_outputs: [B, src_len, hid_dim] — все выходы энкодера
        Возвращает: веса внимания [B, src_len]
        """
        batch_size = encoder_outputs.size(0)
        src_len = encoder_outputs.size(1)

        # Повторяем состояние декодера для каждого временного шага энкодера
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)  # [B, src_len, hid_dim]
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))  # [B, src_len, hid_dim]
        attention = self.v(energy).squeeze(2)  # [B, src_len]
        return torch.softmax(attention, dim=1)


# --- ДЕКОДЕР с вниманием ---
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, attention):
        super(Decoder, self).__init__()
        self.output_dim = output_dim
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)
        # На вход подаём объединённый вектор: embedding + context
        self.rnn = nn.LSTM(emb_dim + hid_dim, hid_dim, n_layers, batch_first=True)
        # Для предсказания токена используем объединение: [output, context, embedded]
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)

    def forward(self, input, hidden, cell, encoder_outputs):
        # input: [B] — один токен (индексы)
        input = input.unsqueeze(1)  # [B, 1]
        embedded = self.embedding(input)  # [B, 1, emb_dim]

        # Вычисляем веса внимания на основе последнего слоя состояния декодера
        a = self.attention(hidden[-1], encoder_outputs)  # [B, src_len]
        a = a.unsqueeze(1)  # [B, 1, src_len]
        # Контекст – взвешенная сумма выходов энкодера
        context = torch.bmm(a, encoder_outputs)  # [B, 1, hid_dim]

        # Объединяем embedding и context
        rnn_input = torch.cat((embedded, context), dim=2)  # [B, 1, emb_dim+hid_dim]

        output, (hidden, cell) = self.rnn(rnn_input, (hidden, cell))
        # output: [B, 1, hid_dim]
        output = output.squeeze(1)  # [B, hid_dim]
        embedded = embedded.squeeze(1)  # [B, emb_dim]
        context = context.squeeze(1)  # [B, hid_dim]
        prediction = self.fc_out(torch.cat((output, context, embedded), dim=1))  # [B, output_dim]
        return prediction, hidden, cell, a.squeeze(1)


# --- Модель Seq2Seq ---
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, src_lengths, trg, teacher_forcing_ratio=0.5):
        """
        src: [B, src_len]
        trg: [B, trg_len] (с токеном <sos> на позиции 0)
        """
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)

        # Получаем выходы энкодера и начальные состояния
        encoder_outputs, hidden, cell = self.encoder(src, src_lengths)

        # Первый токен декодера – <sos>
        input = trg[:, 0]

        for t in range(1, trg_len):
            output, hidden, cell, _ = self.decoder(input, hidden, cell, encoder_outputs)
            outputs[:, t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[:, t] if teacher_force else top1

        return outputs


#########################################
# 3. Функции обучения, перевода и оценки #
#########################################

def train_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    epoch_loss = 0
    for src, trg, src_lengths in dataloader:
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model(src, src_lengths, trg, teacher_forcing_ratio=0.5)
        # output: [B, trg_len, output_dim]
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        trg = trg[:, 1:].reshape(-1)
        loss = criterion(output, trg)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(dataloader)


def translate_sentence(model, sentence, src_vocab, trg_vocab, device, max_len=50):
    """
    Перевод одного предложения.
    sentence: строка с английским текстом.
    Возвращает список токенов перевода.
    """
    model.eval()
    tokens = sentence.lower().split()
    indices = [src_vocab.get(token, src_vocab['<unk>']) for token in tokens]
    src_tensor = torch.tensor(indices, dtype=torch.long, device=device).unsqueeze(0)
    src_len = [len(indices)]

    with torch.no_grad():
        encoder_outputs, hidden, cell = model.encoder(src_tensor, src_len)

    trg_indices = [trg_vocab['<sos>']]

    for _ in range(max_len):
        trg_tensor = torch.tensor([trg_indices[-1]], dtype=torch.long, device=device)
        with torch.no_grad():
            output, hidden, cell, _ = model.decoder(trg_tensor, hidden, cell, encoder_outputs)
        pred_token = output.argmax(1).item()
        trg_indices.append(pred_token)
        if pred_token == trg_vocab['<eos>']:
            break

    inv_trg_vocab = {v: k for k, v in trg_vocab.items()}
    translated_tokens = [inv_trg_vocab.get(idx, '<unk>') for idx in trg_indices]
    return translated_tokens


def evaluate_bleu(model, dataset, device, src_vocab, trg_vocab):
    """
    Вычисляет BLEU score на всём датасете.
    """
    references = []
    hypotheses = []
    for i in range(len(dataset)):
        src_sentence = dataset.src_texts[i]
        trg_sentence = dataset.trg_texts[i].split()
        translation = translate_sentence(model, src_sentence, src_vocab, trg_vocab, device)
        if translation[0] == '<sos>':
            translation = translation[1:]
        if '<eos>' in translation:
            translation = translation[:translation.index('<eos>')]
        references.append([trg_sentence])
        hypotheses.append(translation)
    bleu = corpus_bleu(references, hypotheses)
    return bleu


#########################################
# 4. Основной запуск (обучение, сохранение, оценка) #
#########################################

def main():
    # Параметры
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    BATCH_SIZE = 32
    N_EPOCHS = 10  # Для демонстрации – можно увеличить число эпох
    ENC_EMB_DIM = 256
    DEC_EMB_DIM = 256
    HID_DIM = 512
    N_LAYERS = 1

    # Загружаем датасет
    dataset = TranslationDataset("data/eng-ukr.txt")
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

    INPUT_DIM = len(dataset.src_vocab)
    OUTPUT_DIM = len(dataset.trg_vocab)

    # Инициализируем компоненты модели
    encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, n_layers=N_LAYERS)
    attention = Attention(HID_DIM)
    decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, n_layers=N_LAYERS, attention=attention)
    model = Seq2Seq(encoder, decoder, DEVICE).to(DEVICE)

    optimizer = optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss(ignore_index=dataset.trg_vocab['<pad>'])

    print(f"Начало обучения на {N_EPOCHS} эпох(ах)...")
    for epoch in range(N_EPOCHS):
        start_time = time.time()
        train_loss = train_epoch(model, dataloader, optimizer, criterion, DEVICE)
        end_time = time.time()
        print(f"Эпоха {epoch + 1}/{N_EPOCHS} | Потеря: {train_loss:.3f} | Время: {end_time - start_time:.2f} сек")

    # Сохранение модели и словарей для последующего использования
    torch.save(model.state_dict(), "data/seq2seq_attention_model.pth")
    with open("data/src_vocab.pkl", "wb") as f:
        pickle.dump(dataset.src_vocab, f)
    with open("data/trg_vocab.pkl", "wb") as f:
        pickle.dump(dataset.trg_vocab, f)

    # Оценка качества перевода с помощью BLEU
    bleu_score = evaluate_bleu(model, dataset, DEVICE, dataset.src_vocab, dataset.trg_vocab)
    print(f"BLEU score: {bleu_score * 100:.2f}")


if __name__ == '__main__':
    main()


Начало обучения на 10 эпох(ах)...
Эпоха 1/10 | Потеря: 3.439 | Время: 957.78 сек
Эпоха 2/10 | Потеря: 1.795 | Время: 956.28 сек
Эпоха 3/10 | Потеря: 1.283 | Время: 954.66 сек
Эпоха 4/10 | Потеря: 1.056 | Время: 953.43 сек
Эпоха 5/10 | Потеря: 0.929 | Время: 951.93 сек
Эпоха 6/10 | Потеря: 0.840 | Время: 953.53 сек
Эпоха 7/10 | Потеря: 0.768 | Время: 950.88 сек
Эпоха 8/10 | Потеря: 0.724 | Время: 951.69 сек
Эпоха 9/10 | Потеря: 0.685 | Время: 950.93 сек
Эпоха 10/10 | Потеря: 0.651 | Время: 950.41 сек
BLEU score: 68.57
